In [ ]:
import logging 
logging.basicConfig(level=logging.INFO)
log = logging.getLogger("my-logger")
log.setLevel(logging.INFO)
log.info("Hello World")
log.warning("this is a warning")

In [ ]:
# Test AIReady
import logging
logging.basicConfig(level=logging.INFO)

aiurl = 'http://aiapiplastico-dev.westeurope.cloudapp.azure.com'
isAIready = AIready(f'{aiurl}:5000')

In [ ]:
# Test getPrediction
blob_video_name = '28022020_Boudigau_4_short_480.mov'  
aiurl = 'http://aiapiplastico-dev.westeurope.cloudapp.azure.com'
prediction = getPrediction(blob_video_name,aiurl)

In [ ]:
str(prediction)

In [ ]:
# Import AI + Postgre modules
from modules.ai import ai_ready,get_prediction,get_trash_label,get_trash_time_index,get_trash_time_stamp,get_trash_label,map_label_2_trash_id_PG
from modules.media import get_media_fps
from modules.gps import extract_gpx_from_gopro,gps_point_list,parse_gpx,long_lat_2_shape_point,transform_geo
from modules.postgre import trash_gps,pg_connection_string,pg_open_connection,trash_insert
from tqdm import tqdm

In [ ]:
# Test jsonPrediction
import json
with open('../data/prediction.json') as json_file:
    jsonPred = json.load(json_file)
jsonPred

In [ ]:
frametobox0 = jsonPred['detected_trash'][40]

In [ ]:
timeindex = get_trash_time_index(frametobox0)
mediafps = get_media_fps('/tmp/28022020_Boudigau_4.MP4')
timestamp = get_trash_time_stamp(timeindex,mediafps)
print(timestamp)

In [ ]:
gpxpath = extract_gpx_from_gopro('/tmp/28022020_Boudigau_4.MP4')
gpxdata = parse_gpx(gpxpath)

In [ ]:
gpsPoints = gps_point_list(gpxdata)

In [ ]:
mediafps = get_media_fps('/tmp/28022020_Boudigau_4.MP4')
# Get connection string information from env variables
pgConn_string = pg_connection_string()
# Open pgConnection
pgConnection = pg_open_connection(pgConn_string)
# Create Cursor
pgCursor = pgConnection.cursor()

import logging
logger = logging.getLogger()

row_id_list = []
for prediction in tqdm(jsonPred['detected_trash']):
    try:
        # get trash_gps from gps module
        timeindex = get_trash_time_index(prediction)
        timestamp = get_trash_time_stamp(timeindex,mediafps)
        trash_gps = gpsPoints[timestamp]
        shape_trash_gps = long_lat_2_shape_point(trash_gps)
        geo_2154 = transform_geo(shape_trash_gps)
        geo_2154_trash_gps = {'Time': shape_trash_gps['Time'], 'the_geom': geo_2154,'Latitude':shape_trash_gps['Latitude'],'Longitude':shape_trash_gps['Longitude'], 'Elevation': shape_trash_gps['Elevation']}
        # get trash_type from ai module
        label = get_trash_label(prediction)
        trash_type = map_label_2_trash_id_PG(label)
        # insert
        row_id = trash_insert(geo_2154_trash_gps,trash_type,pgCursor,pgConnection)
        print(row_id)
        row_id_list.append(row_id)
        print(row_id)
    except:
        prediction_id = prediction['id']
        logger.info(f'There was an issue inserting Trash id: {prediction_id} within PostGre')

In [1]:
%env PGSERVER=pgdb-plastico-dev.postgres.database.azure.com
%env PGDATABASE=plastico-dev
%env PGUSERNAME=writer_user@pgdb-plastico-dev
%env PGPWD=SurfWriter!

env: PGSERVER=pgdb-plastico-dev.postgres.database.azure.com
env: PGDATABASE=plastico-dev
env: PGUSERNAME=writer_user@pgdb-plastico-dev
env: PGPWD=SurfWriter!


In [3]:
# Import AI + Postgre modules
from modules.ai import ai_ready,get_prediction,get_trash_label,get_trash_time_index,get_trash_time_stamp,get_trash_label,map_label_2_trash_id_PG
from modules.media import get_media_fps
from modules.gps import extract_gpx_from_gopro,gps_point_list,parse_gpx,long_lat_2_shape_point,transform_geo
from modules.postgre import trash_gps,pg_connection_string,pg_open_connection,trash_insert
from tqdm import tqdm

In [4]:
import json
with open('../data/prediction.json') as json_file:
    jsonPred = json.load(json_file)
gpxpath = extract_gpx_from_gopro('/tmp/28022020_Boudigau_4.MP4')
gpxdata = parse_gpx(gpxpath)
gpsPoints = gps_point_list(gpxdata)
mediafps = get_media_fps('/tmp/28022020_Boudigau_4.MP4')
prediction = jsonPred['detected_trash'][40]
timeindex = get_trash_time_index(prediction)
timestamp = get_trash_time_stamp(timeindex,mediafps)
trash_gps = gpsPoints[timestamp]
shape_trash_gps = long_lat_2_shape_point(trash_gps)
geo_2154 = transform_geo(shape_trash_gps)
geo_2154_trash_gps = {'Time': shape_trash_gps['Time'], 'the_geom': geo_2154,'Latitude':shape_trash_gps['Latitude'],'Longitude':shape_trash_gps['Longitude'], 'Elevation': shape_trash_gps['Elevation']}
# get trash_type from ai module
label = get_trash_label(prediction)
trash_type = map_label_2_trash_id_PG(label)
print(geo_2154_trash_gps)
print(trash_type)

<_sre.SRE_Match object; span=(2554, 2592), match='Stream #0:3(eng): Data: bin_data (gpmd'>
ffprobe version 4.2.2 Copyright (c) 2007-2019 the FFmpeg developers
  built with Apple clang version 11.0.0 (clang-1100.0.33.17)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/4.2.2_2 --enable-shared --enable-pthreads --enable-version3 --enable-avresample --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libbluray --enable-libmp3lame --enable-libopus --enable-librubberband --enable-libsnappy --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-librtmp --enable-libspeex --enable-libsoxr --enable-videotoolbox --disable-libjack --disable-indev=jack
  liba

In [5]:
# Get connection string information from env variables
pgConn_string = pg_connection_string()
# Open pgConnection
pgConnection = pg_open_connection(pgConn_string)
# Create Cursor
pgCursor = pgConnection.cursor()

Connection established


In [6]:
row_id = trash_insert(geo_2154_trash_gps,trash_type,pgCursor,pgConnection)

In [7]:
row_id

'ba9ce862-8faa-43ce-b538-7aafb3de57d6'